In [5]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [6]:
SEASONS = list(range(2016, 2023))

In [40]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [23]:
async def get_html(url, selector, sleep=5, retries=3): #doing async for playwright
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i) #this is so we dont scrape too fast and get banned

        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()  #await is used when we have something async and itll wait until its done and then do this part
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [43]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)#calling beautiful soup with the html we just got and then we are finding all a tags(links)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]

    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [44]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com


FileNotFoundError: [Errno 2] No such file or directory: 'data/standings/NBA_2016_games-october.html'